## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_df.APPLICATION_TYPE.value_counts().loc[lambda x: x<70].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df.CLASSIFICATION.value_counts().loc[lambda x: x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_df.CLASSIFICATION.value_counts().loc[lambda x: x<50].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      391
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [9]:
# Look at AFFILIATION, USE_CASE, ORGANIZATION, STATUS, INCOME_AMT and SPECIAL_CONSIDERATIONS value counts for binning
print(application_df['NAME'].value_counts())
print(application_df['AFFILIATION'].value_counts())
print(application_df['USE_CASE'].value_counts())
print(application_df['ORGANIZATION'].value_counts())
print(application_df['STATUS'].value_counts())
print(application_df['INCOME_AMT'].value_counts())
print(application_df['SPECIAL_CONSIDERATIONS'].value_counts())

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional 

In [10]:
# Drop STATUS and SPECIAL_CONSIDERATIONS columns from dataframe to improve model accuracy
application_df = application_df.drop(columns=['STATUS','SPECIAL_CONSIDERATIONS'])

# # Bin AFFILIATION and USE_CASE columns to reduce outliers
# # Create a list for bins to replace
affiliations_to_replace = list(application_df.AFFILIATION.value_counts().loc[lambda x: x<18000].index)
use_case_to_replace = list(application_df.USE_CASE.value_counts().loc[lambda x: x<20000].index)
org_to_replace = list(application_df.ORGANIZATION.value_counts().loc[lambda x: x<10000].index)
names_to_replace = list(application_df.NAME.value_counts().loc[lambda x: x<300].index)
middle_inc = ['25000-99999', '1-9999', '10000-24999', '100000-499999']
high_inc = ['1M-5M', '10M-50M', '5M-10M', '50M+']

# # Replace in dataframe
for afls in affiliations_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(afls,"Other")
for usc in use_case_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(usc,"Other")
for org in org_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(org,"Other")
for name in names_to_replace:
  application_df['NAME'] = application_df['NAME'].replace(name, "Other")
for minc in middle_inc:
  application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(minc, "mid")
for hinc in high_inc:
  application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(hinc, "high")
    
# # Check to make sure binning was successful
print(application_df['AFFILIATION'].value_counts())
print(application_df['USE_CASE'].value_counts())
print(application_df['ORGANIZATION'].value_counts())
print(application_df['INCOME_AMT'].value_counts())
print(application_df['NAME'].value_counts())

Independent    18480
Other          15819
Name: AFFILIATION, dtype: int64
Preservation    28095
Other            6204
Name: USE_CASE, dtype: int64
Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64
0       24388
mid      8392
high     1519
Name: INCOME_AMT, dtype: int64
Other                                              29662
PARENT BOOSTER USA INC                              1260
TOPS CLUB INC                                        765
UNITED STATES BOWLING CONGRESS INC                   700
WASHINGTON STATE UNIVERSITY                          492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC      408
PTA TEXAS CONGRESS                                   368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC        331
ALPHA PHI SIGMA                                      313
Name: NAME, dtype: int64


In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df["APPLICATION_TYPE"])
class_dummies = pd.get_dummies(application_df['CLASSIFICATION'])
org_dummies = pd.get_dummies(application_df['ORGANIZATION'])
affiliation_dummies = pd.get_dummies(application_df['AFFILIATION'])
use_case_dummies = pd.get_dummies(application_df['USE_CASE'])
name_dummies = pd.get_dummies(application_df['NAME'])
income_dummies = pd.get_dummies(application_df['INCOME_AMT'])

#Add dummy columns to the original df
application_df = pd.concat([application_df, app_dummies, class_dummies, affiliation_dummies, income_dummies, org_dummies, use_case_dummies, name_dummies], axis=1)

#Drop 'APPLICATION_TYPE' and 'CLASSIFICATION' columns
application_df = application_df.drop(columns=["APPLICATION_TYPE", 'CLASSIFICATION', 'AFFILIATION', 'INCOME_AMT', 'USE_CASE', 'ORGANIZATION', 'NAME'])

application_df.head()


,ASK_AMT,IS_SUCCESSFUL,Other,T10,T19,T3,T4,T5,T6,T7,...,Preservation,ALPHA PHI SIGMA,AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,Other,PARENT BOOSTER USA INC,PTA TEXAS CONGRESS,SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,TOPS CLUB INC,UNITED STATES BOWLING CONGRESS INC,WASHINGTON STATE UNIVERSITY
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=46))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="sigmoid"))
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))
nn.add(tf.keras.layers.Dense(units=40, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3760      
                                                                 
 dense_1 (Dense)             (None, 40)                3240      
                                                                 
 dense_2 (Dense)             (None, 40)                1640      
                                                                 
 dense_3 (Dense)             (None, 40)                1640      
                                                                 
 dense_4 (Dense)             (None, 20)                820       
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 11,121
Trainable params: 11,121
Non-traina

In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=250)

Epoch 1/250
804/804 [==============================] - 3s 2ms/step - loss: 0.5423 - accuracy: 0.7283
Epoch 2/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5230 - accuracy: 0.7407
Epoch 3/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5194 - accuracy: 0.7415
Epoch 4/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5184 - accuracy: 0.7441
Epoch 5/250
804/804 [==============================] - 3s 3ms/step - loss: 0.5170 - accuracy: 0.7452
Epoch 6/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5162 - accuracy: 0.7441
Epoch 7/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5151 - accuracy: 0.7462
Epoch 8/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5143 - accuracy: 0.7465
Epoch 9/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5136 - accuracy: 0.7477
Epoch 10/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5135 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5346 - accuracy: 0.7374 - 660ms/epoch - 2ms/step
Loss: 0.5346126556396484, Accuracy: 0.7373760938644409


In [25]:
# Export our model to HDF5 file
df = pd.DataFrame(data=[model_loss, model_accuracy], index=['model loss', 'model accuracy'])
df.to_hdf('AlphabetSoupCharity.h5', key='df', mode='w')
